In [4]:
import os
import numpy as np
import platform
import pandas as pd
import cx_Oracle as odb
import matplotlib.pyplot as plt
import pandas.io.sql as psql
import array

In [5]:
# working dir
if platform.system() == 'Windows':
    os.chdir(os.path.join(os.getenv('HOME'),'Documents', 'Development', 'dst', 'examples', 'abc_db'))
else:
    os.chdir(os.path.join(os.getenv('DST'),'examples', 'abc_db'))
print(os.getcwd())

# plots 
%matplotlib inline
np.set_printoptions(precision=4)
plt.rc('figure', figsize=(10, 8))

/Users/rwk7t/Development/dst/examples/abc_db


In [12]:
# Build connection string
user     = "abc"
sid      = "XE" if platform.system()=='Windows' else "ORCL"
pswd     = "abc"
conn_str =" %s/%s@%s" % (user, pswd, sid)

# Connect to Oracle and test
conn = odb.connect(user, pswd, sid)
if (conn):
    print("Connection successful")
    print(conn.version)
else:
    print("Connection not successful")

#conn.close()

Connection successful
12.1.0.2.0


In [16]:
sql = """\
select p.*, t.name team 
from players p, teams t 
where p.teams_id=t.id\
"""

players_df = psql.read_sql(sql, conn)
players_df.columns = [c.lower() for c in players_df.columns]

In [17]:
players_df

,id,name,starts,goals,image1,image2,image3,teams_id,team
0,1,t1p1,12,7,<cx_Oracle.OBJECT object at 0x10b482c00>,<cx_Oracle.OBJECT object at 0x112739030>,<cx_Oracle.OBJECT object at 0x112739068>,1,team1
1,2,t1p2,2,0,<cx_Oracle.OBJECT object at 0x1127390a0>,<cx_Oracle.OBJECT object at 0x1127390d8>,<cx_Oracle.OBJECT object at 0x112739110>,1,team1
2,3,t1p3,5,11,<cx_Oracle.OBJECT object at 0x112739148>,<cx_Oracle.OBJECT object at 0x112739180>,<cx_Oracle.OBJECT object at 0x1127391b8>,1,team1
3,4,t2p1,5,8,<cx_Oracle.OBJECT object at 0x1127391f0>,<cx_Oracle.OBJECT object at 0x112739228>,<cx_Oracle.OBJECT object at 0x112739260>,2,team2
4,5,t2p2,0,0,<cx_Oracle.OBJECT object at 0x112739298>,<cx_Oracle.OBJECT object at 0x1127392d0>,<cx_Oracle.OBJECT object at 0x112739308>,2,team2
5,6,t2p3,1,0,<cx_Oracle.OBJECT object at 0x112739340>,<cx_Oracle.OBJECT object at 0x112739378>,<cx_Oracle.OBJECT object at 0x1127393b0>,2,team2


In [20]:
obj = players_df.iloc[0].image1
obj

In [21]:
dir(obj)

['__class__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'type']

In [22]:
obj.type, obj.DTYPE, obj.NDIM, obj.DIM1, obj.DIM2, obj.DIM3, obj.AVG, obj.STD

(<cx_Oracle.ObjectType ABC.ARRAYINFO>,
 'uint8',
 3.0,
 743.0,
 508.0,
 3.0,
 108.590448737649,
 43.8245216500585)

In [44]:
sql = """
SELECT player.id AS player_id,
  player.name,
  player.starts,
  player.goals,
  player.teams_id,
  player.image1.avg,
  player.image1.std,
  player.image1.ndim,
  player.image1.dim1,
  player.image1.dim2,
  player.image1.dim3,
  player.image1.dim4,
  player.image1.dtype,
  player.image1.value
FROM players player
"""
df  = psql.read_sql(sql, conn)
df.columns = [c.lower() for c in df.columns]
df

,player_id,name,starts,goals,teams_id,image1.avg,image1.std,image1.ndim,image1.dim1,image1.dim2,image1.dim3,image1.dim4,image1.dtype,image1.value
0,1,t1p1,12,7,1,108.590449,43.824522,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
1,2,t1p2,2,0,1,108.424215,43.737101,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
2,3,t1p3,5,11,1,108.423759,43.741233,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
3,4,t2p1,5,8,2,108.472490,43.796886,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
4,5,t2p2,0,0,2,108.498431,43.829951,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
5,6,t2p3,1,0,2,108.453264,43.802637,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...


In [47]:
for i, idx in enumerate(df.index):
    ndim  = df.iloc[i]['image1.ndim' ]
    dim1  = df.iloc[i]['image1.dim1' ]
    dim2  = df.iloc[i]['image1.dim2' ]
    dim3  = df.iloc[i]['image1.dim3' ] 
    dtype = df.iloc[i]['image1.dtype']
    blob  = df.iloc[i]['image1.value']
    a     = np.array(array.array('B', blob.read()), dtype=np.uint8).reshape((dim1, dim2, dim3))
    df.ix[i]['image1'] = a
    
del(df['image1.ndim', 'image1.dim1'])
df

AttributeError: 'numpy.ndarray' object has no attribute 'array'

In [40]:
for r in df:
    print(r)

player_id
name
starts
goals
teams_id
image1.avg
image1.std
image1.ndim
image1.dim1
image1.dim2
image1.dim3
image1.dim4
image1.dtype
image2.avg
image2.std
image2.ndim
image2.dim1
image2.dim2
image2.dim3
image2.dim4
image2.dtype
image3.avg
image3.std
image3.ndim
image3.dim1
image3.dim2
image3.dim3
image3.dim4
image3.dtype
